# Team DatenWelle

## Keyword merging with FuzzyWuzzy

This notebook loads the data from JSON format and performs some keyword cleaning and merging misspelled duplicates with fuzzyWuuzy package 

In [1]:
#!git pull
#!git status
#!git add 2-anya-keywords_RapidFuzz.ipynb 2-anya-keywords_to_dataframe_vector.ipynb #2-anya-keywords_FuzzyWuzzy.ipynb #1-anya_exploratory_analysis.ipynb
#!git commit -m 'playing with RapidFuzz (its much faster than FuzzyWuzzy!)'
#!git push
#!pip install -r ../requirements.txt

#after installed new libraries
#!pip freeze > requirements.txt
#!git add requirements.txt 
#!git add out_dedupl_100323.csv test.csv
#!git commit -m 'added output files f fuzzy wuzzy dedupe'
#!git commit -m 'added library rapidfuzy'
#!git push 

In [2]:
import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from fuzzywuzzy.process import dedupe
import functools
from rapidfuzz import process as pr

In [3]:
# Opening JSON file
f = open('../data/raw/CMS_2010_to_June_2022_ENGLISH.json')
  
# returns JSON object as 
# a dictionary
data = json.load(f)

df = pd.DataFrame.from_dict(data)

#print(df.head())

#### Finding the subset of the data for 1 Jan 2019 - 1 Jan 2020 based on lastModifiedDate

In [4]:
df = df.sort_values(by='lastModifiedDate') #sort dataframe

datetimes = pd.to_datetime(df['lastModifiedDate'])
df['ts_lastModifiedDate']=datetimes
#df.iloc[ts_start]['ts_lastModifiedDate']

#find start index for subset 2019-2022
ts_start=datetimes[(datetimes > pd.Timestamp(year=2019, month=1, day=1).tz_localize('utc')) 
          & (datetimes < pd.Timestamp(year=2019, month=1, day=2).tz_localize('utc'))].min()
print(ts_start)
#find end date for subset 2019-2022
ts_end=datetimes[(datetimes > pd.Timestamp(year=2022, month=1, day=1).tz_localize('utc')) 
          & (datetimes < pd.Timestamp(year=2022, month=1, day=2).tz_localize('utc'))].min()
print(ts_end)

start_date=datetimes[datetimes == ts_start]
end_date=datetimes[datetimes == ts_end]

#find index for the chosen start and end dates
start_index=start_date.index[0]
print(start_index)
df[df.index == start_date.index[0]]

end_index=end_date.index[0]
print(end_index)
df[df.index == end_date.index[0]]

df_subset=df[start_index:end_index]


2019-01-01 03:57:28.904000+00:00
2022-01-01 02:35:51.098000+00:00
60278
150367


In [5]:
#df_subset=df_subset[:100]

33830

In [ ]:
len(df_subset)

### Keywords exploration


In [ ]:
df['keywords'] # is keywords in dictionary

In [ ]:
df['keywords'].isna().sum()


In [8]:
#create series of keywords sets
def get_keywords(row):
    if row is None:
        return None
    else:
        res_set = set()
        for name_dict in row:
            res_set.add(name_dict['name'])
        return res_set

#df['keywords'].apply(get_keywords)

In [9]:
#extract individual keywords from the sets of sets

# should work but it is very slow for now with current gpus
# 10000 articles in 7 seconds
# df_subset (90090 articles) runs in 10 minutes 10 seconds

# sets=df_subset['keywords'].apply(get_keyword1) #full dataset
sets=df_subset['keywords'].apply(get_keywords)  #2019-2021 subset
#sets=sets[0:10000] #10000 articles
 
kw=functools.reduce(set.union, sets)

In [10]:
len(kw)

32704

In [11]:
uni_kw=(pd.DataFrame(kw, columns = ['keyword']))

In [12]:
len(list(uni_kw['keyword']))

32704

# Cleaning

## 1 step putting everything in lower case

In [28]:
substring = ', '

list_multikw = [kw for kw in keywords_clean if substring in kw] # keywords that did not get split
new_keywords = [kw.split(substring) for kw in list_multikw] # make a list of new keywords (the splited multi kw)

# Flatten list of list of new keywords
flat_new_keywords = [item for sublist in new_keywords for item in sublist]
flat_new_keywords.remove('') # remove empty values

# Remove the non-seperated keywords
for el in list_multikw:
    keywords_clean.remove(el)

# Add the seperated ones
keywords_clean = keywords_clean + flat_new_keywords

print('after splitting:', len(keywords_clean)) # number is higher because split long keyowrds into multiplev

after splitting: 30404


### Cleaning 3: remove '\u2002' and '.' and '\xa0'

In [29]:
# Identify all single keywords that contain commas, splits them in mutliple keywords and saves this in a list
new_keywords = []
list_kw_todel = []
substring = ', '
for i,s in enumerate(keywords_clean):
    if substring in s:
        #print('row', i, ':',  s)
        list_kw_todel.append(s)
        new_keywords.append(keywords_clean[i].split(substring))

flat_new_keywords = [item for sublist in new_keywords for item in sublist]

# Remove the non-seperated keywords
for el in list_kw_todel:
    keywords_clean.remove(el)

# Add the seperated ones
keywords_clean = keywords_clean + flat_new_keywords

print(len(keywords_clean))

30405


In [ ]:
# # another way to extract individual keywords from the sets of sets that doesn't crash kernel is interrupted
# # so it might be more stable when later applied to the entire dataset

# #runs for 10000 articles in 5 seconds
# #runs for df_subset in  11 min 16 sec 

# sets=df_subset['keywords'].apply(get_keywords)
# #sets=sets[0:10000]
# from tqdm import tqdm
# def get_unique_keywords(sets):
#     result_set = set()
#     for row_set in tqdm(sets.values):
#         #result_set.union(row_set)
#         result_set = result_set.union(row_set)
#     return result_set

# unique_keywords = get_unique_keywords(sets)



In [30]:
#write the set of DW keywords before fuzzyWuzzy into the file
# pd.Series(list(unique_keywords)).to_csv('../data/interim/out_2019-2021_keywords_before_FuzzyWuzzy.csv')
kw=keywords_clean
pd.Series(list(kw)).to_csv('../data/interim/out_2019-2021_keywords_before_FuzzyWuzzy_1503.csv')

In [ ]:
#unique_keywords=kw

In [35]:
# #load set of DW keywords before fuzzyWuzzy into the file
#uni_kw=pd.read_csv('../data/interim/out_2019-2021_keywords_before_FuzzyWuzzy.csv') 
uni_kw=pd.read_csv('../data/interim/out_2019-2021_keywords_before_FuzzyWuzzy_1503.csv', names = ['ind', 'keyword'], header=0)

In [37]:
uni_kw.head()

,ind,keyword
0,0,markus braun
1,1,product development
2,2,eu energy transition
3,3,non-swimmers
4,4,graft


In [ ]:
set(uni_kw['0'])

In [ ]:
#unique_keywords=set(uni_kw['0'])

In [ ]:
unique_keywords

In [ ]:
# sets_10000=sets

### Playing with FuzzyWuzzy

In [ ]:
help(process)

In [ ]:
help(fuzz.token_set_ratio)

In [ ]:
# ## !!VERY SLOW!!! 
# # Took 186 minutes to run for 10000 articles
# # took 35 minutes for 10000 keywords

# #fuzzy.process.dedupe function returns a list without duplicates. by default it is using 70% similarity ratio
# #to explore similarity ratio for individual words use fuzzy.process.extract i.e. process.extract('angela merkel',unique_keywords,limit=20)

# print(len(unique_keywords))
# #ded_kw=dedupe(unique_keywords)
# #ded_kw=dedupe(unique_keywords, threshold = 90)
# ded_kw=dedupe(list(unique_keywords)[:10000], threshold = 90)
# print(len(ded_kw))

In [ ]:
# #write the deduplicated keywords into the file
# #pd.Series(list(ded_kw)).to_csv('../data/interim/out_dedupl_10k_articles_only_100323.csv')
# pd.Series(list(ded_kw)).to_csv('../data/interim/out_dedupl_10k_kw_only_140323_threshold_90.csv')
# #pd.Series(list(ded_kw)).to_csv('../data/interim/out_dedupl_2019-2021_articles_only_100323_threshold_90.csv')

In [ ]:
# # #read from csv isntead of running DEDUP
# #ded_kw=pd.read_csv('../data/interim/out_dedupl_10k_articles_only_100323.csv')
# ded_kw=pd.read_csv('../data/interim/out_dedupl_2019-2021_articles_only_100323.csv')

### Trying out the RapidFuzz https://maxbachmann.github.io/RapidFuzz/Usage/process.html 

In [ ]:

#from rapidfuzz import process, fuzz #
help(rapidfuzz.process.cdist)

In [ ]:
from rapidfuzz import process as pr
import pandas as pd

In [ ]:
# #load set of DW keywords before fuzzyWuzzy into the file
uni_kw=pd.read_csv('../data/interim/out_2019-2021_keywords_before_FuzzyWuzzy.csv') 
#unique_keywords=uni_kw

In [ ]:
(list(uni_kw['0']))

In [ ]:

# 20k by 20k is analysed in  19.6 sec!!!
# for 32704 (2years of data) by 32704 ratios are calculated in 26 sec cut off 70

#ratio_array=pr.cdist(unique_keywords,unique_keywords,score_cutoff = 70)
ratio_array= pr.cdist(list(uni_kw['0']),list(uni_kw['0']),score_cutoff = 70)

In [ ]:
df_array = pd.DataFrame(ratio_array, columns = list(uni_kw['0']), index=list(uni_kw['0']))

In [ ]:
df_array

In [ ]:
#from rapidfuzz import process as pr

#process.extract("new york jets", choices, scorer=fuzz.WRatio, limit=2)
#rapidfuzz.process.extract('angela merkel',unique_keywords,scorer=fuzz.WRatio,limit=40)
pr.extract('angela merkel',unique_keywords,  limit=40)

In [ ]:
deduplicated #=ded_kw

### Exploring ratio of similarity for individual  keywords

In [ ]:
process.extract('angela merkel',unique_keywords,limit=40)


In [ ]:
process.extract('planetary defense conference',unique_keywords,limit=40)


In [ ]:
process.extract('Chosen Soren',unique_keywords,limit=20)

In [ ]:
process.extract('Sex pistols',unique_keywords,limit=20)

In [ ]:
process.extract('UEFA',unique_keywords,limit=40)

In [ ]:
process.extract('UAE',unique_keywords,limit=40)

In [ ]:
process.extract('United Arab Emirates',unique_keywords,limit=40)

In [ ]:
process.extract('war in Ukraine',unique_keywords,limit=40)

In [ ]:
process.extract('UK',unique_keywords,limit=40)

In [ ]:
process.extract('United Kingdom',unique_keywords,limit=40)

In [ ]:
# #example from the fuzzywuzzy tutorial on token ratio
# Str1 = "The supreme court case of Nixon vs The United States"
# Str2 = "Nixon v. United States"
# Ratio = fuzz.ratio(Str1.lower(),Str2.lower())
# Partial_Ratio = fuzz.partial_ratio(Str1.lower(),Str2.lower())
# Token_Sort_Ratio = fuzz.token_sort_ratio(Str1,Str2)
# Token_Set_Ratio = fuzz.token_set_ratio(Str1,Str2)
# print(Ratio)
# print(Partial_Ratio)
# print(Token_Sort_Ratio)
# print(Token_Set_Ratio)


### "Putting back" merged clean keywords into the dataframe

In [ ]:
list_kw=list(unique_keywords)[0]

In [ ]:
df_subset['keywordStrings'][0]

In [ ]:
# i=0
# len(df_subset['keywordStrings'][i])

In [ ]:
# #n=0
# df_subset['keywordStrings'][i][n]

In [ ]:
df_subset

In [ ]:
# ## speeding up put_clean_kw_into_df
# # 1) apply to every over each line in dataframe
# # 2) loop over each keyword in the line
# # 3) find process.extractOne a substitute from deduplicated list
# # 4) create a new column in dataframe with merged keywords

# def put_clean_kw_into_df_fast(dataframe):
#     #df.applymap(lambda x: len(str(x)))

#     dataframe.applymap

#     return answer


In [ ]:
# # 1) loop over each line in dataframe
# # 2) loop over each keyword in the line
# # 3) find process.extractOne a substitute from deduplicated list
# # 4) create a new column in dataframe with merged keywords


# def put_clean_kw_into_df(dataframe):
#     i = 0
#     sample_line = []
#     sample_ratio = []
#     while i <= 5: #len(dataframe): # 10:
#         print(i)
#         n = 0
#         line_wr = []
#         ratio_line_wr = []
#         #print(line_wr)
#         while n < len(dataframe[i]):
#             #print(process.extractOne(df_subset['keywordStrings'][i][n],deduplicated)) #print word and ratio 
#             line_wr.append(process.extractOne(dataframe[i][n], deduplicated)[0])
#             ratio_line_wr.append(process.extractOne(dataframe[i][n], deduplicated)[1])
#             n = n+1

#         sample_line.append(line_wr)
#         sample_ratio.append(ratio_line_wr)

#         i = i+1
#     return sample_line
        
    
# df_lines=put_clean_kw_into_df(df_subset['keywordStrings'])



In [ ]:
df_subset['keywordStrings'][:5]

In [ ]:
df_lines

In [ ]:
pd.Series(df_lines).to_csv('../data/interim/out_df_lines_10k_kw_2years_subset_mess.csv')


In [ ]:
print(lines)

In [ ]:
df_subset['keywordStrings'][i]


In [ ]:
#figure out list comprehension later
# i=0
# def put_clean_kw_into_df(dataframe):
#     return [process.extractOne(dataframe[n], deduplicated)[0] for dataframe[n] in dataframe]

# put_clean_kw_into_df(df_subset['keywordStrings'][i])